# Model Inference and Evaluation on Validation Set. Inference on Test Set.

Evaluation On Validation Set:
1. Perform inference from base (mistral instruct v0.3) and fine tuned model on validation set.
2. Compute BLEU, Rouge, BERTScore between both models and human responses.

For Test Set:
1. Perform inference from base (mistral instruct v0.3) and fine tuned model on test set generated using GPT-4o (see notebook 'pqr').
2. We output the models' responses to a file.
3. In notebook 'xyz', we use GPT-5 as LLM-as-a-Judge to compare base (mistral instruct v0.3) and the fine tuned model.


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U git+https://github.com/huggingface/trl.git
!pip install flash-attn --no-build-isolation
!pip install quanto
!pip install evaluate bert-score
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 39.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 62.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.8.3-cp312-cp312-linux_x86_64.whl size=256040057 sha256=f25da18657a87fc83dc1bfb8b7751b82246e9db355510226b674fd437c34b5fb
  Stored i

In [ ]:
# Mount GDrive - will prompt authentication
from google.colab import drive
drive.mount('/content/drive')

## Get HF tokenizer
with open("/content/drive/MyDrive/ColabNotebooks/ParentPalAI/data/hftoken.txt") as f:
    HF_TOKEN = f.read().strip()


BASE_MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"


Mounted at /content/drive


In [ ]:
import re
from datasets import load_dataset

# build prompt
# We modify the system instructions a bit
def build_prompt(prompt):
  return f'''<s>[INST] You are a friendly parenting companion who gives helpful advice like a fellow parent would. You sound warm and practical — not robotic or formal.
Keep your answers under 150 words.
Stay focused on the user’s question.
Avoid Reddit-style responses like 'Edit:', smiley faces, or overly casual tone.
User's Prompt: {prompt} [/INST]'''


# Clean input
def get_q_and_a(text):
  qasplit = text.split("[/INST]")
  q = qasplit[0]
  q = q.replace('[INST]', '').replace('<s>', '').strip()
  if len(qasplit)>1:
    a = qasplit[1]
    a = a.replace('</s>', '').strip()
  else:
    a = None
  return (q, a)

# Clean Output
def clean_text(text):
    # Cut anything after 'Source'
    text = re.split(r'\b(Source:)', text)[0].strip()
    # Remove the last incomplete sentence (if any)
    sentences = re.split(r'(?<=[.!?])\s+', text)
    if sentences and not text.strip().endswith(('.', '!', '?')):
        sentences = sentences[:-1]  # remove the last incomplete part
    return ' '.join(sentences)


In [ ]:
# load validation set of 100 samples
# Load training and validation sets as hugging face datasets
validation_df = load_dataset('json', data_files='/content/drive/MyDrive/ColabNotebooks/ParentPalAI/data/reddit_dataset_val_100.jsonl', split='train')
print (f"Number of validation examples: {len(validation_df)}")


In [ ]:
# load test set of GPT-generated 100 samples
# Load training and validation sets as hugging face datasets
test_df = load_dataset('json', data_files='/content/drive/MyDrive/ColabNotebooks/ParentPalAI/data/gpt_reddit_test_samples.jsonl', split='train')
print (f"Number of test examples: {len(test_df)}")


Generating train split: 0 examples [00:00, ? examples/s]

Number of test examples: 100


In [ ]:
# Load Eval Metrics
import evaluate
import numpy as np
import pandas as pd


bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Load Base model and Evaluate it


**BLEU Score**

{'bleu': 0.006949776517672412, 'precisions': [0.16613720271314503, 0.014029618082618862, 0.0022713374683323143, 0.0004406451044328897], 'brevity_penalty': 1.0, 'length_ratio': 1.588949522510232, 'translation_length': 11647, 'reference_length': 7330}

**ROUGE SCORE**

{'rouge1': np.float64(0.17742945129434573), 'rouge2': np.float64(0.01612920672542411), 'rougeL': np.float64(0.09766495290266788), 'rougeLsum': np.float64(0.1039609198966609)}


BERTScore:
**bold text**- BERT avg precision: 0.8333736592531205
- BERT avg recall: 0.8439619988203049
- BERT avg recall: 0.8385502642393112




In [ ]:
# LOAD THE BASE MODEL IN 4-BIT PRECISION WITH DOUBLE QUANTIZATION

import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # loads base model in 4-bit precision
    bnb_4bit_use_double_quant=True, # double quantization saves VRAM
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.bfloat16,
    attn_implementation="flash_attention_2", # FA2 is fastest on A100
    token=HF_TOKEN # login to hugging face
)




config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# load tokenizer - we will infer for 1 prompt at a time - so no padding needed
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL_ID,
    add_bos_token=False,
    add_eos_token=False,
    token = HF_TOKEN
)
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Evaluating the Base Model on the Validation Set

In [ ]:
# Run inference on all prompts in the validation set.
# And compute BLEU, ROUGE, BERTScore with human responses.
import time

predictions = []
references = []
ques_list = []
for i, ex in enumerate(validation_df):
  print (f"Running for example {i+1}")
  st = time.time()
  ques, human_answer = get_q_and_a(ex["text"])
  with torch.no_grad():
    inputs = tokenizer(build_prompt(ques), return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.15,
        eos_token_id=tokenizer.eos_token_id,
    )
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=False)
    _, ai_answer = get_q_and_a(raw_output)
    ai_answer = clean_text(ai_answer)
    predictions.append(ai_answer)
    references.append(human_answer)
    ques_list.append(ques)
    print (f"Time taken for example {i+1}: {(time.time()-st)/60.0}")



In [ ]:
blue_results = bleu.compute(predictions=predictions, references=references)
print (blue_results)

rouge_results = rouge.compute(predictions=predictions, references=references)
print (rouge_results)

bert_results = bertscore.compute(predictions=predictions, references=references, lang='en')
print (bert_results)

print (f"BERT avg precision: {np.mean(bert_results['precision'])}")
print (f"BERT avg recall: {np.mean(bert_results['recall'])}")
print (f"BERT avg f1: {np.mean(bert_results['f1'])}")


In [ ]:
# save predictions
validation_df_pd = validation_df.to_pandas()
validation_df_pd['user_prompt'] = ques_list
validation_df_pd['ai_answer'] = predictions
validation_df_pd['human_answer'] = references
validation_df_pd["bertscore_f1"] = bert_results["f1"]
validation_df_pd["bertscore_precision"] = bert_results["precision"]
validation_df_pd["bertscore_recall"] = bert_results["recall"]
validation_df_pd.to_json('/content/drive/MyDrive/ColabNotebooks/ParentPalAI/data/reddit_dataset_val_100_base_response.jsonl', orient='records', lines=True)


## Inferencing the Base Model on GPT Generated Test Set

This will be used to evaluate using LLM-as-a-Judge

In [ ]:
# Run inference on all prompts in the validation set.
# And compute BLEU, ROUGE, BERTScore with human responses.
import time

predictions = []
ques_list = []
for i, ex in enumerate(test_df):
  print (f"Running for example {i+1}")
  st = time.time()
  ques, _ = get_q_and_a(ex["text"])
  with torch.no_grad():
    inputs = tokenizer(build_prompt(ques), return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.15,
        eos_token_id=tokenizer.eos_token_id,
    )
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=False)
    _, ai_answer = get_q_and_a(raw_output)
    ai_answer = clean_text(ai_answer)
    predictions.append(ai_answer)
    ques_list.append(ques)
    print (f"Time taken for example {i+1}: {(time.time()-st)/60.0}")



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Running for example 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Time taken for example 1: 0.23644356727600097
Running for example 2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Time taken for example 97: 0.20438342094421386
Running for example 98


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Time taken for example 98: 0.20314055681228638
Running for example 99


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Time taken for example 99: 0.20354623794555665
Running for example 100
Time taken for example 100: 0.20428076187769573


In [ ]:
# save predictions
test_df_pd = test_df.to_pandas()
test_df_pd['user_prompt'] = ques_list
test_df_pd['ai_answer'] = predictions
test_df_pd.to_json('/content/drive/MyDrive/ColabNotebooks/ParentPalAI/data/gpt_reddit_test_samples_base_response.jsonl', orient='records', lines=True)


# Load the Fine Tuned Model and Evaluate it

In [ ]:
# Load the adapter weights for the fine tuned model
from peft import PeftModel

run_name = "parentpalai"
output_dir = "/content/drive/MyDrive/ColabNotebooks/ParentPalAI/model/" + run_name
model = PeftModel.from_pretrained(model, output_dir+"/checkpoint-1600")

model.config.use_cache = True
model.eval() # sets the model into evaluation mode. Disables dropout, layernorm, etc.



PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_pro

In [ ]:
# # Lets also push the model to HF
# new_name = 'PretendParentAI'
# model.push_to_hub(new_name, token=HF_TOKEN)
# tokenizer.push_to_hub(new_name, token=HF_TOKEN)



In [ ]:
## Run inference on example prompt
prompt = '''How to respond to 'I don't love you' -
My 2.5 year-old daughter has pretty much always been a daddy's girl. It's understandable because her dad is great, and much more patient than I am. But in the last month or so it's escalated to her saying 'I don't want you, Mama', 'I don't like you, Mama', and 'I don't love you, Mama', sometimes followed by 'I love Dada'. I find this really hard to deal with and I don't think I'm responding in the best way. Sometimes I try to ignore it, but sometimes I make a sad face and ask why, but of course she's not able to articulate that at this age.

Yesterday I broke down crying because she'd been saying it a lot. Her dad encouraged her to give me a hug. Today she was doing it again and her dad said no, we don't say that. Then I said, 'why do you say that? It makes me sad because I love you very much. Remember when Mama was crying, why do you think she was crying?' She didn't seem to have a response.

I know it's problematic to make children feel like they're responsible for a parent's emotional well-being. But is it appropriate for them to know they've made you sad and why? What is the best way to respond to these kinds of comments from a toddler?
'''
inputs = tokenizer(build_prompt(prompt), return_tensors="pt").to(model.device)

# top_p: determines nucleus sampling
## Instead of picking only the single most likely next token, model samples from a pool of tokens whose probabilities sum to 90%.
## Prevents the model from always picking the most boring “safe” token.
## Lower top_p (e.g., 0.7) = more focused, deterministic.
## Higher top_p (e.g., 0.95–1.0) = more diverse, creative, but risk of nonsense.
## 0.9 is a balanced default for natural-sounding answers.


# repetition_penalty: Discourages the model from repeating the same words/phrases.
## It re-weights probabilities: tokens that already appeared get penalized.
## 1.0 = no penalty (default).
## >1.0 = stronger penalty → less repetition.
## Too high (like >1.5) = model may avoid repeating even when it should (“the the” becomes impossible).
## 1.15 is a gentle nudge to stop the “looping answers” problem.


with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150, # lets go for shorter answers
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.15,
        eos_token_id=tokenizer.eos_token_id,
    )

raw_output = tokenizer.decode(outputs[0], skip_special_tokens=False)
raw_output = raw_output.split("</s>")[0]

ques, ans = get_q_and_a(raw_output)
print ("\nQuestion")
print (ques)

print ("\nCleaned Answer")
ans_clean = clean_text(ans)
print (ans_clean)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Question
You are a friendly parenting companion who gives helpful advice like a fellow parent would. You sound warm and practical — not robotic or formal.
Keep your answers under 150 words.
Stay focused on the user’s question.
Avoid Reddit-style responses like 'Edit:', smiley faces, or overly casual tone.
User's Prompt: How to respond to 'I don't love you' -
My 2.5 year-old daughter has pretty much always been a daddy's girl. It's understandable because her dad is great, and much more patient than I am. But in the last month or so it's escalated to her saying 'I don't want you, Mama', 'I don't like you, Mama', and 'I don't love you, Mama', sometimes followed by 'I love Dada'. I find this really hard to deal with and I don't think I'm responding in the best way. Sometimes I try to ignore it, but sometimes I make a sad face and ask why, but of course she's not able to articulate that at this age.

Yesterday I broke down crying because she'd been saying it a lot. Her dad encouraged her t

## Evaluating Fine Tuned Model on Validation Set

**BLEU (Bilingual Evaluation Understudy):**
It's the overlap between machine translation output and reference output. More suited for short tasks like machine transalation. The list contains precision for unigrams, bigrams, trigrams, 4-grams. The BLEU score is very poor but that's because this is not an appropriate metric for this task.

Length is 1.73x the reference. So, we know that our model generates longer responses. Brevity 1 means no penalty applied for brevity because generated outputs are not too short.

{'bleu': 0.0062420097799231605, 'precisions': [0.17395070477990393, 0.016906103659020556, 0.0016001280102408192, 0.00032260666182756673], 'brevity_penalty': 1.0, 'length_ratio': 1.732469304229195, 'translation_length': 12699, 'reference_length': 7330}

**ROUGE (Recall-Oriented Understudy for Gisting Evaluation)**
Measures the overlap between the generated text and human-written reference texts. Suitable for tasks like translation and summarization. It's less strict than BLEU. ROUGE-1: Measures the overlap of unigrams (individual words). ROUGE-2: Measures the overlap of bigrams (two-word sequences). ROUGE-L: Focuses on the longest common subsequence to capture sentence-level structure and semantic similarity.

{'rouge1': np.float64(0.2068109926799131), 'rouge2': np.float64(0.021508314761075942), 'rougeL': np.float64(0.10574799893449807), 'rougeLsum': np.float64(0.1058553007217165)}

- ROUGE-1 i.e. unigram overlap is 20% which is moderate
- ROUGE-2 i.e. consecutive word overlap is 2.15% which is very low
- ROUGE-L measures the longest common subsequence. 0.10 is very low.


**BERTScore**
For each sample, embeds each token in prediction and reference using BERT. Then computes cosine similarity between each token in the prediction and reference.

Precision: How well do tokens in the prediction match those in the reference? For each prediction token, find the most similar reference token and get the cosine similarity.

Recall: How well do tokens in the reference match those in the prediction? For each reference token, find the most similar prediction token and get the cosine similarity.

F1: Harmonic mean of precision and recall.

We average prediction, recall, and F1 across all samples, to get the following:

- BERT avg precision: 0.8323308438062668
- BERT avg recall: 0.8462447559833527
- BERT avg f1: 0.8391281771659851

In [ ]:
# Run inference on all prompts in the validation set.
# And compute BLEU, ROUGE, BERTScore with human responses.
import time

predictions = []
references = []
ques_list = []
for i, ex in enumerate(validation_df):
  print (f"Running for example {i+1}")
  st = time.time()
  ques, human_answer = get_q_and_a(ex["text"])
  with torch.no_grad():
    inputs = tokenizer(build_prompt(ques), return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.15,
        eos_token_id=tokenizer.eos_token_id,
    )
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=False)
    _, ai_answer = get_q_and_a(raw_output)
    ai_answer = clean_text(ai_answer)
    predictions.append(ai_answer)
    references.append(human_answer)
    ques_list.append(ques)
    print (f"Time taken for example {i+1}: {(time.time()-st)/60.0}")


In [ ]:
blue_results = bleu.compute(predictions=predictions, references=references)
print (blue_results)

rouge_results = rouge.compute(predictions=predictions, references=references)
print (rouge_results)

bert_results = bertscore.compute(predictions=predictions, references=references, lang='en')
print (bert_results)

print (f"BERT avg precision: {np.mean(bert_results['precision'])}")
print (f"BERT avg recall: {np.mean(bert_results['recall'])}")
print (f"BERT avg f1: {np.mean(bert_results['f1'])}")


In [ ]:
# save predictions
validation_df_pd = validation_df.to_pandas()
validation_df_pd['user_prompt'] = ques_list
validation_df_pd['ai_answer'] = predictions
validation_df_pd['human_answer'] = references
validation_df_pd["bertscore_f1"] = bert_results["f1"]
validation_df_pd["bertscore_precision"] = bert_results["precision"]
validation_df_pd["bertscore_recall"] = bert_results["recall"]
validation_df_pd.to_json('/content/drive/MyDrive/ColabNotebooks/ParentPalAI/data/reddit_dataset_val_100_ft_response.jsonl', orient='records', lines=True)


## Inferencing Fine Tuned Model on GPT Generated Test Set

This will be used to evaluate using LLM-as-a-Judge

In [ ]:
# Run inference on all prompts in the validation set.
# And compute BLEU, ROUGE, BERTScore with human responses.
import time

predictions = []
ques_list = []
for i, ex in enumerate(test_df):
  print (f"Running for example {i+1}")
  st = time.time()
  ques, _ = get_q_and_a(ex["text"])
  with torch.no_grad():
    inputs = tokenizer(build_prompt(ques), return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.15,
        eos_token_id=tokenizer.eos_token_id,
    )
    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=False)
    _, ai_answer = get_q_and_a(raw_output)
    ai_answer = clean_text(ai_answer)
    predictions.append(ai_answer)
    ques_list.append(ques)
    print (f"Time taken for example {i+1}: {(time.time()-st)/60.0}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Running for example 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Time taken for example 1: 0.3536958773930868
Running for example 2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Time taken for example 97: 0.35326074759165443
Running for example 98


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Time taken for example 98: 0.3535870631535848
Running for example 99


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Time taken for example 99: 0.3519122322400411
Running for example 100
Time taken for example 100: 0.3508267879486084


In [ ]:
# save predictions
test_df_pd = test_df.to_pandas()
test_df_pd['user_prompt'] = ques_list
test_df_pd['ai_answer'] = predictions
test_df_pd.to_json('/content/drive/MyDrive/ColabNotebooks/ParentPalAI/data/gpt_reddit_test_samples_ft_response.jsonl', orient='records', lines=True)
